# Rue du commerce web crawler


In [13]:
import requests
import unittest
from bs4 import BeautifulSoup


In [14]:
website_prefixe = "https://www.rueducommerce.fr/recherche/pc-portable-"
website_suffixe = "?sort=remise&universe=MC-3540&view=grid"


In [15]:
def _handle_request_result_and_build_soup(request_result):
    if request_result.status_code == 200:
        html_doc =  request_result.text
        soup = BeautifulSoup(html_doc,"html.parser")
        return soup


In [10]:
def _convert_string_to_float(string):
    temp = string.strip()
    if temp[0]== '(' : # small negative percentage
        return float(temp[1:-2])
    if temp[-1]== '-' : # -- NA value
        return 0.0
    temp = ''.join(temp.split(',')) # remove thousand's comma
    return float(temp)


In [11]:
page = "dell"

In [18]:
res = requests.get("https://www.darty.com/nav/recherche?s=prix_desc&npk=1&text=dell&fa=756")

In [24]:
soup = _handle_request_result_and_build_soup(res)

In [31]:
l1 = soup.find_all("p", class_="darty_prix_barre_remise darty_small separator_top")
list(map(lambda x : x.text, l1))

['- 34%',
 '- 40%',
 '- 50%',
 '- 43%',
 '- 43%',
 '- 6%',
 '- 27%',
 '- 23%',
 '- 23%',
 '- 23%',
 '- 26%']

In [33]:
page = "acer"

In [35]:
res = requests.get("https://www.darty.com/nav/recherche?s=prix_desc&npk=1&text=acer&fa=756")
soup = _handle_request_result_and_build_soup(res)
l1 = soup.find_all("p", class_="darty_prix_barre_remise darty_small separator_top")
list(map(lambda x : x.text, l1))

['- 22%', '- 23%', '- 23%', '- 25%', '- 25%', '- 26%']